In [1]:
import numpy as np
import cv2

In [2]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [3]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    # ------------TODO---------------
    # compute point cloud from depth image
    # for-loop is not allowed!!
    # ------------TODO --------------

    r,c=depth.shape
    cx,cy=K[0][2],K[1][2]
    alpha,beta=K[0][0],K[1][1]
    pc=np.zeros((r,c,3))

    x_axis = np.linspace(0,c-1,c)
    y_axis = np.linspace(0,r-1,r)
    X, Y = np.meshgrid(x_axis, y_axis)
    x=((X-cx)/alpha*depth).reshape(r,c,1)
    y=((Y-cy)/beta*depth).reshape(r,c,1)
    z=depth.reshape(r,c,1)
    pc=np.concatenate((x,y,z),axis=2)
    pc=pc[seg>0]
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [4]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------
def one_way_chamfer_distance(pc1,pc2):
    cd=0
    n1,n2=pc1.shape[0],pc2.shape[0]
    arr1=np.tile(pc1,(n2,1))
    arr2=np.repeat(pc2,n1,axis=0)
    dis=np.sqrt(np.sum((arr1-arr2)**2,axis=1)).reshape(n2,n1)
    dis=np.mean(np.min(dis,axis=0))
    return [dis]

one_way_CD = one_way_chamfer_distance(partial_pc_sampled,full_pc_sampled)
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', one_way_CD)

one way chamfer distance:  [0.010366844052845488]
